In [1]:
import os, sys
import numpy as np
from multihist import Histdd, Hist1d
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'   # enable if you have a retina display
import pandas as pd
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import UnivariateSpline
from scipy.special import expi

pd.options.mode.chained_assignment = None        # default='warn'
import warnings
warnings.filterwarnings('ignore')

In [2]:
dsets = pd.read_pickle('/home/zhut/data/Delayed/data/run_names_v6.6.5_100.pkl')

In [3]:
def plt_config(title = None, xbounds = None, ybounds = None, xlabel = None, ylabel = None, colorbar = False, sci = True):
    if sci:
        plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.title(title)
    plt.xlim(xbounds)
    plt.ylim(ybounds)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if isinstance(colorbar,str):
        plt.colorbar(label = colorbar)
    elif colorbar:
        plt.colorbar(label = 'Number of entries')
    else:
        pass

In [4]:
pax_version = '6.6.5'
your_own_path = '/home/zhut/data/Delayed/data/pickles/pax_v%s_elist' % pax_version

In [5]:
df_copy = []
for filename in os.listdir(your_own_path):
    if '.pkl' in filename:
        df_copy.append(pd.read_pickle(os.path.join(your_own_path,filename)))
    if len(df_copy) > 10:
        break
df_copy = pd.concat(df_copy)

In [6]:
#print(df_copy.columns)
not_in_use_col = []
not_in_use_col += [col for col in df_copy.columns if ('nearest' in col) or ('next' in col)]
not_in_use_col += [col for col in df_copy.columns if ('largest_' in col)]
not_in_use_col.remove('largest_other_s2')
df_copy.drop(not_in_use_col,axis = 1,inplace = True)
print(df_copy.columns)

Index(['cs1', 'cs2', 'drift_time', 'event_duration', 'event_number',
       'event_time', 'largest_other_s2', 'previous_1e5pe_event',
       'previous_1e6pe_event', 'previous_3e5pe_event', 'previous_busy',
       'previous_busy_off', 'previous_busy_on', 'previous_event',
       'previous_hev', 'previous_hev_off', 'previous_hev_on',
       'previous_muon_veto_trigger', 'previous_s2_area', 'run_number', 's1',
       's1_area_fraction_top', 's1_range_50p_area', 's2',
       's2_area_fraction_top', 's2_range_50p_area', 's2aft_low_lim',
       's2aft_up_lim', 'x', 'y', 'z'],
      dtype='object')


In [ ]:
from numpy import sqrt, log


plt.rcParams['figure.figsize'] = (16, 10)
fig = plt.figure()

if False:"""
sel_clean = lambda df:df[(df.previous_event > 10e6) &
                         (df.event_number != 0) & 
                         (df.previous_s2_area > 0) & 
                         (df.previous_hev > 80e6) &
                         (df.previous_muon_veto_trigger > 80e6) &
                         (np.log10(df.previous_s2_area**0.25*np.exp(-df.previous_event/5e6)) < -4) &
                         (df.drift_time > 5e3) &
                         (df.largest_other_s2 < ses2_size_cut_ori(df.s2))
                        ]
df = sel_clean(df_copy)

#Main S2 area fraction top cut
df['s2aft_up_lim'] = (0.6177399420527526 + 3.713166211522462e-08 * df.s2 + 0.5460484265254656 / np.log(df.s2))
df['s2aft_low_lim'] = (0.6648160611018054 - 2.590402853814859e-07 * df.s2 - 0.8531029789184852 / np.log(df.s2))
df = df[df.s2_area_fraction_top<df.s2aft_up_lim]
df = df[df.s2_area_fraction_top>df.s2aft_low_lim]



def ses2_size_cut_ori(s2_area):
    linear_0 = s2_area *0.01 + 90
    linear_1 = s2_area * 0.025 + 766
    fermi_dirac_coef_0 = 1 / (np.exp((s2_area - 26000) * 6e-4) + 1) 
    fermi_dirac_coef_1 = 1 / (np.exp((26000 - s2_area) * 6e-4) + 1)
    return linear_0*fermi_dirac_coef_0+linear_1*fermi_dirac_coef_1"""


#df = df_copy
df = quality_control_cuts(df_copy)
#####################################################################
ax = fig.add_subplot(221)
mh = Histdd(df.drift_time, df.s2_area_fraction_top,
            bins = (np.linspace(0,0.69e6,100), np.linspace(0,1,100))
           )
mh.plot(log_scale = True, cmap=plt.get_cmap('viridis'), alpha = 1.0 )


#####################################################################
ax = fig.add_subplot(222)
mh = Histdd(df.drift_time, df.s2_range_50p_area,
            bins = (np.linspace(0,0.69e6,100), np.linspace(0,10e3,100))
           )
mh.plot(log_scale = True, cmap=plt.get_cmap('viridis'), alpha = 1.0 )


#####################################################################
ax = fig.add_subplot(223)
mh = Histdd(np.log10(df.s2), np.log10(df.largest_other_s2),
            bins = (np.linspace(2,6,100), np.linspace(1,5,100))
           )
mh.plot(log_scale = True, cmap=plt.get_cmap('viridis'), alpha = 1.0 )


#####################################################################
ax = fig.add_subplot(224)
mh = Histdd(df.x**2 + df.y**2, df.z,
            bins = (np.linspace(0,2300,100), np.linspace(-103,3,100))
           )
mh.plot(log_scale = True, cmap=plt.get_cmap('viridis'), alpha = 1.0 )

plt.show()

In [11]:
def quality_control_cuts(df):
    
    def ses2_size_cut_ori(s2_area):
        linear_0 = s2_area *0.01 + 90
        linear_1 = s2_area * 0.025 + 766
        fermi_dirac_coef_0 = 1 / (np.exp((s2_area - 26000) * 6e-4) + 1) 
        fermi_dirac_coef_1 = 1 / (np.exp((26000 - s2_area) * 6e-4) + 1)
        return linear_0*fermi_dirac_coef_0+linear_1*fermi_dirac_coef_1

    sel_dtime = lambda df:df[(df.event_number >= 10) &
                             (df.previous_event > 10e6) &
                             (df.previous_hev > 80e6) & (df.previous_muon_veto_trigger > 80e6) &
                             (np.log10(df.previous_s2_area**0.25*np.exp(-df.previous_event/5e6)) < -4)
                            ]
    df = sel_dtime(df)
    
    sel_usual = lambda df:df[(df.drift_time > 5e3) &
                             (df.largest_other_s2 < ses2_size_cut_ori(df.s2)) &
                             (df.x ** 2 + df.y **2 < 2200)
                            ]
    df = sel_usual(df)
    # just applying a loose fv cuts

    df['s2aft_up_lim'] = (0.6177399420527526 + 3.713166211522462e-08 * df.s2 + 0.5460484265254656 / np.log(df.s2))
    df['s2aft_low_lim'] = (0.6648160611018054 - 2.590402853814859e-07 * df.s2 - 0.8531029789184852 / np.log(df.s2))
    df = df[df.s2_area_fraction_top < df.s2aft_up_lim]
    df = df[df.s2_area_fraction_top > df.s2aft_low_lim]

    sel_tnext = lambda df:df[(df.next_hev_on > df.next_event) &
                             (df.next_muon_veto_trigger > df.next_event)
                            ]
    
    
    return (df)

In [82]:
class data_process():

    def __init__(self, run):
        self.run = run

    data_folder = '/home/zhut/data/Delayed/data'

    input_path = []

    output_path = [
        data_folder + '/pickles/pax_v6.6.5_elist_all',
        data_folder + '/pickles/pax_v6.6.5_elist_cut',
        data_folder + '/pickles/pax_v6.6.5_peak_raw',
        data_folder + '/pickles/pax_v6.6.5_peak_combine'
                  ]
            

    def check(self):
        name = self.run + '.pkl'
        
        for i, path in enumerate(self.output_path):
            if os.path.isfile(os.path.join(path,name)):
                pass
            else:
                return False, path
        
        return True, ''

In [83]:
for name in dsets.name.values:
    dp = data_process(name)
    flag, path = dp.check()
    if flag:
        pass
        #print ('%s ready' %name)
    else:
        print ('%s not yet in %s' %(name, path.split('/')[-1]))

170307_0831 not yet in pax_v6.6.5_peak_combine
170310_1816 not yet in pax_v6.6.5_elist_all
170311_0919 not yet in pax_v6.6.5_peak_raw
170403_2120 not yet in pax_v6.6.5_peak_combine
170404_0621 not yet in pax_v6.6.5_peak_combine
170418_0148 not yet in pax_v6.6.5_peak_combine
170514_0825 not yet in pax_v6.6.5_peak_combine
170515_1036 not yet in pax_v6.6.5_elist_all
170515_1136 not yet in pax_v6.6.5_elist_all
170515_1336 not yet in pax_v6.6.5_elist_all
170515_1436 not yet in pax_v6.6.5_elist_all
170515_1637 not yet in pax_v6.6.5_elist_all
170515_1837 not yet in pax_v6.6.5_elist_all
170515_1938 not yet in pax_v6.6.5_elist_all
170515_2038 not yet in pax_v6.6.5_elist_all
170515_2238 not yet in pax_v6.6.5_elist_all
170516_0239 not yet in pax_v6.6.5_elist_all
170516_0339 not yet in pax_v6.6.5_elist_all
170516_0640 not yet in pax_v6.6.5_elist_all
170516_0740 not yet in pax_v6.6.5_elist_all
